In [84]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

In [85]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

## import data set

In [86]:
dm = pd.read_csv('pbp_merged.csv')
dm = dm.drop('Unnamed: 0', axis=1)

- keep only regular season games and drop irrelevant observations. Exclude overtime and shootouts.

In [87]:
dm = dm[dm['GameNumber'] <= 21230]

In [88]:
dm = dm[dm['EventType']!='STOP']
dm = dm[dm['EventType']!='EISTR']
dm = dm[dm['EventType']!='EIEND']

In [89]:
dm = dm[dm['Period'] <= 3]
dm = dm[dm['Period'] >= 1]

In [90]:
dm.shape

(310113, 44)

### reshape the data set from wide to long.

In [91]:
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

In [92]:
a = [col for col in dm.columns if 'VPlayer' in col]
b = [col for col in dm.columns if 'HPlayer' in col]
c = [col for col in dm.columns if 'VPosition' in col]
d = [col for col in dm.columns if 'HPosition' in col]
dm = pd.lreshape(dm, {'VPlayer' : a, 'HPlayer' : b, 'VPosition' : c, 'HPosition': d})

In [93]:
dm.shape

(1796745, 24)

In [94]:
dm.columns

Index(['AdvantageType', 'EventDetail', 'EventNumber', 'EventTimeFromTwenty',
       'EventTimeFromZero', 'EventType', 'GameDate', 'GameNumber', 'HTeamCode',
       'Length', 'PenaltyType', 'Period', 'PlayerName', 'PlayerNumber',
       'Season', 'ShotResult', 'ShotType', 'TeamCode', 'VTeamCode', 'Zone',
       'VPlayer', 'HPlayer', 'VPosition', 'HPosition'],
      dtype='object')

In [95]:
dm = dm.rename(columns={'PlayerNumber': 'EventPlayerNumber', 'TeamCode': 'EventTeamCode', 'PlayerName': 'EventPlayerName' })
dm = dm[['Season', 'GameNumber', 'GameDate', 'Period', 'AdvantageType', 'Zone', 'EventNumber', 'EventType', 'EventDetail', 'EventTeamCode', 'EventPlayerNumber', 'EventPlayerName', 'EventTimeFromZero', 'EventTimeFromTwenty', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition', 'ShotType', 'ShotResult', 'Length', 'PenaltyType']]

In [96]:
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

In [97]:
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')
dm['EventPlayerNumber'] = dm['EventPlayerNumber'].fillna('TEAM')

In [98]:
dm.to_csv('play_by_play.csv', index='False', sep=',')

#### create new data set and keep variables: 
 - (a) game number.
 - (b) visitor team information.
 - (c) home team information.

In [99]:
df = dm[['Season', 'GameNumber', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition']]
df = df.sort_values(['Season', 'GameNumber'], ascending=[True, True])
df.head()

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition
0,2010,20001,MTL,11.0,C,TOR,37.0,C
310113,2010,20001,MTL,21.0,R,TOR,9.0,R
620126,2010,20001,MTL,57.0,L,TOR,11.0,L
930061,2010,20001,MTL,26.0,D,TOR,3.0,D
1239931,2010,20001,MTL,75.0,D,TOR,22.0,D


In [100]:
dg = pd.read_csv('season_game_data.csv')
dg = dg.drop('Unnamed: 0', axis=1)

- merge season_games data on play by play data

In [101]:
df = pd.merge(df, dg, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='left')
df.head()                         

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,11.0,C,TOR,37.0,C,2,3,1,3,2,TOR,MTL
1,2010,20001,MTL,21.0,R,TOR,9.0,R,2,3,1,3,2,TOR,MTL
2,2010,20001,MTL,57.0,L,TOR,11.0,L,2,3,1,3,2,TOR,MTL
3,2010,20001,MTL,26.0,D,TOR,3.0,D,2,3,1,3,2,TOR,MTL
4,2010,20001,MTL,75.0,D,TOR,22.0,D,2,3,1,3,2,TOR,MTL


- reshape the data to have home and visitor team observatons under the same coloumns. 

In [102]:
a = [col for col in df.columns if 'Player' in col]
b = [col for col in df.columns if 'Position' in col]
c = [col for col in df.columns if 'TeamCode' in col]
d = [col for col in df.columns if 'GF' in col]
e = [col for col in df.columns if 'GA' in col]
df = pd.lreshape(df, {'PlayerNumber' : a, 'PlayerPosition' : b, 'TeamCode' : c, 'GF' : d, 'GA' : e })

In [103]:
df.head()

,GD,GameNumber,LossTeam,Season,WinTeam,PlayerNumber,PlayerPosition,TeamCode,GF,GA
0,1,20001,MTL,2010,TOR,11.0,C,MTL,2,3
1,1,20001,MTL,2010,TOR,21.0,R,MTL,2,3
2,1,20001,MTL,2010,TOR,57.0,L,MTL,2,3
3,1,20001,MTL,2010,TOR,26.0,D,MTL,2,3
4,1,20001,MTL,2010,TOR,75.0,D,MTL,2,3


- import player rankings

In [104]:
dp = pd.read_csv('player_rank_manual.csv')
dp = dp.drop('Unnamed: 0', axis=1)

#### display each player by team per game. Drop duplicates.

In [105]:
dw = pd.merge(df, dp, on=['Season', 'TeamCode', 'PlayerNumber', 'PlayerPosition'], how='left')

- create column that displays the position and roster count by team per game. 

In [106]:
dw = dw[dw.PlayerPosition!='G']
#dw = dw[dw.GameNumber==20001]
dw = dw.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerNumber'])
dw['RosterCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode'])['PlayerNumber'].transform('count')
#dw = dw[dw.RosterCount==18]
dw['Position'] = dw.apply(lambda x: 'D' if x['PlayerPosition']=='D' else 'F', 1)
dw['PositionCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode', 'Position'])['PlayerNumber'].transform('count')
dw.head()

,GD,GameNumber,LossTeam,Season,WinTeam,PlayerNumber,PlayerPosition,TeamCode,GF,GA,Rank,RosterCount,Position,PositionCount
0,1,20001,MTL,2010,TOR,11.0,C,MTL,2,3,2,18.0,F,12.0
1,1,20001,MTL,2010,TOR,21.0,R,MTL,2,3,1,18.0,F,12.0
2,1,20001,MTL,2010,TOR,57.0,L,MTL,2,3,2,18.0,F,12.0
3,1,20001,MTL,2010,TOR,26.0,D,MTL,2,3,2,18.0,D,6.0
4,1,20001,MTL,2010,TOR,75.0,D,MTL,2,3,2,18.0,D,6.0


- count the amount of forwards and defensemen by team per game.

In [107]:
dw['FCount'] = dw.apply(lambda x: x['PositionCount'] if x['Position']=='F' else np.NaN, 1)
dw['DCount'] = dw.apply(lambda x: x['PositionCount'] if x['Position']=='D' else np.NaN, 1)

In [108]:
dw['FCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['FCount'].apply(lambda x: x.ffill().bfill())
dw['DCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['DCount'].apply(lambda x: x.ffill().bfill())
dw.head()

,GD,GameNumber,LossTeam,Season,WinTeam,PlayerNumber,PlayerPosition,TeamCode,GF,GA,Rank,RosterCount,Position,PositionCount,FCount,DCount
0,1,20001,MTL,2010,TOR,11.0,C,MTL,2,3,2,18.0,F,12.0,12.0,6.0
1,1,20001,MTL,2010,TOR,21.0,R,MTL,2,3,1,18.0,F,12.0,12.0,6.0
2,1,20001,MTL,2010,TOR,57.0,L,MTL,2,3,2,18.0,F,12.0,12.0,6.0
3,1,20001,MTL,2010,TOR,26.0,D,MTL,2,3,2,18.0,D,6.0,12.0,6.0
4,1,20001,MTL,2010,TOR,75.0,D,MTL,2,3,2,18.0,D,6.0,12.0,6.0


## keep games that have only 12 F and 6 D per team!!!!

In [109]:
dw = dw.groupby(['Season', 'GameNumber']).filter(lambda x: ((x['FCount'] == 12) & (x['DCount'] == 6)).all())

In [110]:
dw.shape

(36540, 16)

In [111]:
dw.to_csv('team_roster_player_rank.csv', index='False')

- display the mean ranking per position by team for each game.

In [112]:
dw = dw.groupby(['Season', 'GameNumber', 'TeamCode', 'Position', 'GF', 'GA', 'RosterCount', 'PositionCount', 'FCount', 'DCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dw.head(10)

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank
0,2010,20001,MTL,D,2,3,18.0,6.0,12.0,6.0,TOR,MTL,1.833333
1,2010,20001,MTL,F,2,3,18.0,12.0,12.0,6.0,TOR,MTL,1.833333
2,2010,20001,TOR,D,3,2,18.0,6.0,12.0,6.0,TOR,MTL,1.833333
3,2010,20001,TOR,F,3,2,18.0,12.0,12.0,6.0,TOR,MTL,1.833333
4,2010,20002,PHI,D,3,2,18.0,6.0,12.0,6.0,PHI,PIT,1.666667
5,2010,20002,PHI,F,3,2,18.0,12.0,12.0,6.0,PHI,PIT,1.583333
6,2010,20002,PIT,D,2,3,18.0,6.0,12.0,6.0,PHI,PIT,1.500000
7,2010,20002,PIT,F,2,3,18.0,12.0,12.0,6.0,PHI,PIT,1.583333
8,2010,20003,CAR,D,4,3,18.0,6.0,12.0,6.0,CAR,MIN,1.833333
9,2010,20003,CAR,F,4,3,18.0,12.0,12.0,6.0,CAR,MIN,1.750000


In [113]:
dw.shape

(4060, 13)

- create columns for team win and team loss. 

In [114]:
dw['TeamWin'] =  dw.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dw['TeamLos'] =  dw.apply(lambda x: 1 if x['TeamCode']!=x['WinTeam'] else 0, 1)
dw.head()

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos
0,2010,20001,MTL,D,2,3,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,0,1
1,2010,20001,MTL,F,2,3,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,0,1
2,2010,20001,TOR,D,3,2,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,1,0
3,2010,20001,TOR,F,3,2,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,1,0
4,2010,20002,PHI,D,3,2,18.0,6.0,12.0,6.0,PHI,PIT,1.666667,1,0


- display games played, games won, games loss, goals for and goals against by team for the season.

In [115]:
dw['GP'] = dw.groupby(['Season', 'Position', 'TeamCode'])['GameNumber'].transform('count')
dw['GW'] = dw.groupby(['Season', 'Position', 'WinTeam'])['TeamWin'].transform('sum')
dw['GL'] = dw.groupby(['Season', 'Position', 'LossTeam'])['TeamLos'].transform('sum')
dw['GF'] = dw.groupby(['Season', 'Position', 'TeamCode'])['GF'].transform('sum')
dw['GA'] = dw.groupby(['Season', 'Position', 'TeamCode'])['GA'].transform('sum')
dw.head()

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL
0,2010,20001,MTL,D,188,181,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31
1,2010,20001,MTL,F,188,181,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31
2,2010,20001,TOR,D,195,218,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31
3,2010,20001,TOR,F,195,218,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31
4,2010,20002,PHI,D,235,207,18.0,6.0,12.0,6.0,PHI,PIT,1.666667,1,0,72,41,31


- create columns with the mean ranking for forward and defenseman by team per game.

In [116]:
dw['Rank_F'] = dw.apply(lambda x: x['Rank'] if x['Position']=='F' else np.NaN, 1)
dw['Rank_D'] = dw.apply(lambda x: x['Rank'] if x['Position']=='D' else np.NaN, 1)
dw['Rank_F'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['Rank_F'].apply(lambda x: x.ffill().bfill())
dw['Rank_D'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['Rank_D'].apply(lambda x: x.ffill().bfill())
dw.head()

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,Rank_F,Rank_D
0,2010,20001,MTL,D,188,181,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333
1,2010,20001,MTL,F,188,181,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333
2,2010,20001,TOR,D,195,218,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333
3,2010,20001,TOR,F,195,218,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333
4,2010,20002,PHI,D,235,207,18.0,6.0,12.0,6.0,PHI,PIT,1.666667,1,0,72,41,31,1.583333,1.666667


- compute the mean per position by team for the season.

In [117]:
dw['Mean_F']= dw.groupby(['Season', 'TeamCode'])['Rank_F'].transform('mean')
dw['Mean_D']= dw.groupby(['Season', 'TeamCode'])['Rank_D'].transform('mean')
dw.head()

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,Rank_F,Rank_D,Mean_F,Mean_D
0,2010,20001,MTL,D,188,181,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333,1.734069,1.825980
1,2010,20001,MTL,F,188,181,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333,1.734069,1.825980
2,2010,20001,TOR,D,195,218,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333,1.834524,1.888095
3,2010,20001,TOR,F,195,218,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333,1.834524,1.888095
4,2010,20002,PHI,D,235,207,18.0,6.0,12.0,6.0,PHI,PIT,1.666667,1,0,72,41,31,1.583333,1.666667,1.592593,1.571759


- display the quantity of wins and losses per team ( roster of 12 forwards and 6 defensemen)

In [118]:
dw['L'] = dw.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dw['W'] = dw.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)
dw.head()

,Season,GameNumber,TeamCode,Position,GF,GA,RosterCount,PositionCount,FCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,Rank_F,Rank_D,Mean_F,Mean_D,L,W
0,2010,20001,MTL,D,188,181,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333,1.734069,1.825980,31,37
1,2010,20001,MTL,F,188,181,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,0,1,68,34,31,1.833333,1.833333,1.734069,1.825980,31,37
2,2010,20001,TOR,D,195,218,18.0,6.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333,1.834524,1.888095,36,34
3,2010,20001,TOR,F,195,218,18.0,12.0,12.0,6.0,TOR,MTL,1.833333,1,0,70,34,31,1.833333,1.833333,1.834524,1.888095,36,34
4,2010,20002,PHI,D,235,207,18.0,6.0,12.0,6.0,PHI,PIT,1.666667,1,0,72,41,31,1.583333,1.666667,1.592593,1.571759,31,41


- compute win and loss percent by team. Drop duplicate observations.

In [119]:
dx = dw[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'Mean_F', 'Mean_D']]
dx = dx.drop_duplicates(['Season', 'TeamCode'])
dx['WinPc'] = dx['W']/ dx['GP']
dx['LossPc'] = dx['L']/ dx['GP']
dx = dx[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'Mean_F', 'Mean_D']]
dx.head()

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,Mean_F,Mean_D
0,2010,MTL,68,37,31,188,181,0.544118,0.455882,1.734069,1.825980
2,2010,TOR,70,34,36,195,218,0.485714,0.514286,1.834524,1.888095
4,2010,PHI,72,41,31,235,207,0.569444,0.430556,1.592593,1.571759
6,2010,PIT,71,40,31,204,180,0.563380,0.436620,1.642019,1.561033
8,2010,CAR,76,38,38,222,218,0.500000,0.500000,1.751096,1.804825


- rank teams based on win percent, mean forwards and mean defensemen. 

In [120]:
dx['Rank_W'] = dx.groupby(['Season'])['WinPc'].rank(ascending=False)
dx['Rank_F'] = dx.groupby(['Season'])['Mean_F'].rank(ascending=True)
dx['Rank_D'] = dx.groupby(['Season'])['Mean_D'].rank(ascending=True)
dx = dx.sort_values(['Season', 'Rank_W', 'Rank_F', 'Rank_D'], ascending=[True, True, True, True])
dx.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,Mean_F,Mean_D,Rank_W,Rank_F,Rank_D
70,2010,VAN,73,48,25,238,175,0.657534,0.342466,1.602740,1.600457,1.0,4.0,8.0
22,2010,SJ,65,41,24,206,158,0.630769,0.369231,1.519231,1.535897,2.0,1.0,2.0
36,2010,BOS,76,45,31,228,176,0.592105,0.407895,1.603070,1.530702,3.0,5.0,1.0
30,2010,DET,68,40,28,220,201,0.588235,0.411765,1.636029,1.546569,4.0,8.0,3.0
28,2010,ANA,65,38,27,186,181,0.584615,0.415385,1.608974,1.648718,5.0,6.0,9.0
102,2010,WSH,72,42,30,190,169,0.583333,0.416667,1.613426,1.571759,6.0,7.0,6.0
68,2010,LA,70,40,30,194,178,0.571429,0.428571,1.602381,1.559524,7.0,3.0,4.0
4,2010,PHI,72,41,31,235,207,0.569444,0.430556,1.592593,1.571759,8.0,2.0,7.0
6,2010,PIT,71,40,31,204,180,0.563380,0.436620,1.642019,1.561033,9.0,9.0,5.0
42,2010,NYR,73,41,32,223,177,0.561644,0.438356,1.712329,1.680365,10.0,13.0,12.0


**Crosby played 41 games, Malkin 43 and Staal 42 for Pittsburgh**

In [121]:
dx.to_csv('season_team_roster.csv', index='False')

In [122]:
dx.shape

(30, 14)